In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "Resources/purchase_data.csv"

In [3]:
### Player Count
original_df = pd.read_csv(file)
original_df.head()
total_players = original_df["SN"].nunique()
print(f'Total players:',total_players)

Total players: 576


In [4]:
### Purchasing Analysis (Total)
unique_items = original_df["Item Name"].nunique()

average_price = original_df["Price"].mean()
average_price = round(average_price,2)

total_purchases = original_df["SN"].count()

total_revenue = original_df['Price'].sum()
print(f'Unique items:', unique_items)
print(f'Average item price:', average_price)
print(f'Total number of purchases:', total_purchases)
print(f'Total revenue:', total_revenue)


Unique items: 179
Average item price: 3.05
Total number of purchases: 780
Total revenue: 2379.77


In [5]:
### Gender Demographics
gender_count = original_df.groupby('Gender')['SN'].nunique()

female_count = gender_count["Female"]
female_perc = "{:.2%}".format(female_count / total_players)

male_count = gender_count['Male']
male_perc = "{:.2%}".format(male_count / total_players)

other_count = gender_count['Other / Non-Disclosed']
other_perc = "{:.2%}".format(other_count/ total_players)

print(f'Total count', gender_count,'\n')

print(f'Female percentage:',female_perc)
print(f'Male percentage:',male_perc)
print(f'Other percentage:',other_perc)

Total count Gender
Female                    81
Male                     484
Other / Non-Disclosed     11
Name: SN, dtype: int64 

Female percentage: 14.06%
Male percentage: 84.03%
Other percentage: 1.91%


In [6]:
### Purchasing Analysis (Gender)
gender_purchase_count = original_df.groupby('Gender')

total_purchase_count = gender_purchase_count['Price'].count()

average_purchase = round(gender_purchase_count['Price'].mean(),2)

total_purchase_value = gender_purchase_count['Price'].sum()


gender_df = pd.concat([total_purchase_count, average_purchase, total_purchase_value], axis=1)

gender_df.columns=["Purchase Count", "Average Purchase Price", "Total Purchase Value"]



gender_df['Average Purchase Total per Person by Gender'] = round(gender_df["Total Purchase Value"]/gender_count, 2)
gender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person by Gender
Gender,,,,
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


In [10]:
### Age Demographics (7-45)
age_bins = [0]
age_max = original_df['Age'].max()
for i in range(10,age_max + 4,5):
    age_bins.append(i)
group_names = ['0-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41+']
original_copy_df = original_df
original_copy_df["Age Group"] = pd.cut(original_copy_df['Age'], age_bins, labels = group_names)
age_group_sort = original_copy_df.groupby('Age Group')

age_count = original_copy_df.groupby('Age Group')['SN'].nunique()

age_purchase_count = age_group_sort['Price'].count()


age_average_purchase = round(age_group_sort['Price'].mean(),2)

age_purchase_value = age_group_sort['Price'].sum()


age_df = pd.concat([age_purchase_count, age_average_purchase, age_purchase_value], axis=1)


age_df.columns=["Purchase Count", "Average Purchase Price", "Total Purchase Value"]




age_df['Average Purchase Total per Person by Age'] = round(age_df["Total Purchase Value"]/age_count, 2)
age_df



,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person by Age
Age Group,,,,
0-10,32,3.40,108.96,4.54
11-15,54,2.90,156.60,3.82
16-20,200,3.11,621.56,4.14
21-25,325,3.02,981.64,4.23
26-30,77,2.88,221.42,3.75
31-35,52,2.99,155.71,4.21
36-40,33,3.40,112.35,4.32
41+,7,3.08,21.53,3.08


In [8]:
### Top Spenders
player_sort = original_df.groupby('SN')
top_5_spenders = player_sort['Price'].sum().sort_values(ascending = False).head(5)
top_spenders_df = original_df.join(top_5_spenders, on='SN', how='right', rsuffix="top")
top_spenders_cut = top_spenders_df[['SN', 'Price', 'Pricetop']]
grouped_top_df = top_spenders_cut.groupby('SN')
grouped_sum = grouped_top_df["Price"].sum()
grouped_count = grouped_top_df["Price"].count()
grouped_mean = round(grouped_top_df["Price"].mean(),2)

top_df = pd.concat([grouped_count, grouped_mean, grouped_sum], axis=1)
top_df.columns=["Purchase Count", "Average Purchase Price", "Total Purchase Value"]
top_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Chamjask73,3,4.61,13.83
Idastidru52,4,3.86,15.45
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10
Lisosia93,5,3.79,18.96


In [98]:
### Most Popular Items
item_sort = original_df.groupby('Item Name')
top_5_items = item_sort['Item Name'].count().sort_values(ascending = False).head(5)
top_item_df = original_df.join(top_5_items, on='Item Name', how='right', rsuffix="top")
top_item_cut = top_item_df[['Item Name','Item ID','Price']]
grouped_item_df = top_item_cut.groupby("Item Name")
purchase_count = grouped_item_df['Item Name'].count()
purchase_count.columns=["Purchase Count"]
total_purchase_value = grouped_item_df['Price'].sum()
average_price = round(grouped_item_df['Price'].mean(),2)
top_item_df = pd.concat([purchase_count,total_purchase_value,average_price], axis=1)
top_item_df.columns=["Purchase Count",'Total Purchase Value','Price']
top_item_df2 = top_item_df.merge(original_df, on='Item Name', how='inner', suffixes=' y',)
top_item_df2
top_item_df3 = top_item_df2[['Item ID', 'Item Name', 'Purchase Count','Price ', 'Total Purchase Value']]
top_item_df3.drop_duplicates()

,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
0,145,Fiery Glass Crusader,9,4.58,41.22
9,92,Final Critic,13,4.61,59.99
22,82,Nirvana,9,4.90,44.10
31,178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
43,132,Persuasion,9,3.22,28.99


In [112]:
### Most Profitable Items
profit_sort=original_df.groupby('Item Name')
profit_5 = profit_sort['Price'].sum().sort_values(ascending = False).head(5)
original_copy_df['Max Profit']=profit_5

#dropped_na = original_copy_df.dropna()
#dropped_na

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Max Profit, dtype: float64